In [5]:
import time  # Import the time module

# Start timing
start_time = time.time()

import numpy as np
from scipy.spatial import ConvexHull
import networkx as nx
from shapely.geometry import LineString

# Generate random points on a sphere for the outer polytope
num_points = 100
phi = np.pi * (3. - np.sqrt(5.))  # golden angle in radians

y = np.linspace(1 - 1 / num_points, 1 / num_points - 1, num_points)
radius = np.sqrt(1 - y * y)

theta = phi * np.arange(num_points)

x = np.cos(theta) * radius
z = np.sin(theta) * radius

vertices_outer = np.vstack([x, y, z]).T

# Compute the convex hull for the outer polytope
hull_outer = ConvexHull(vertices_outer)

# Scale down for the inner polytope with 0.2 distance
scale_factor = 0.8
center_outer = np.mean(vertices_outer, axis=0)
vertices_inner = (vertices_outer - center_outer) * scale_factor + center_outer

# Compute the convex hull for the inner polytope
hull_inner = ConvexHull(vertices_inner)

# Initialize the graph
G = nx.Graph()

# Add nodes for the outer and inner vertices
for vertex in vertices_outer:
    G.add_node(tuple(vertex))
for vertex in vertices_inner:
    G.add_node(tuple(vertex))

# Function to add edges of the polytope
def add_edges_polytope(graph, vertices, max_distance=np.inf):
    num_vertices = len(vertices)
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            u, v = tuple(vertices[i]), tuple(vertices[j])
            distance = np.linalg.norm(vertices[i] - vertices[j])
            if distance <= max_distance:
                graph.add_edge(u, v, weight=distance)

# Add edges for the outer and inner polytopes with a distance threshold
add_edges_polytope(G, vertices_outer, max_distance=0.5)
add_edges_polytope(G, vertices_inner, max_distance=0.5)

# Define source and end points
source = (-0.8, -0.8, -0.8)
end = (0.8, 0.8, 0.8)

# Add source and end points as nodes
G.add_node(source)
G.add_node(end)

# Function to add edges from a point to polytope vertices with more connections
def add_edges_without_intersection(graph, point, vertices, hull, max_connections=None, max_distance=np.inf):
    distances = []
    for vertex in vertices:
        distance = np.linalg.norm(np.array(point) - np.array(vertex))
        distances.append((distance, vertex))
    
    # Sort by distance and limit the number of connections if max_connections is set
    distances.sort()
    if max_connections is not None:
        distances = distances[:max_connections]

    for distance, vertex in distances:
        if distance <= max_distance:
            weight = LineString([point, vertex]).length
            graph.add_edge(point, tuple(vertex), weight=weight)

# Add more edges for source and end points
add_edges_without_intersection(G, source, vertices_outer, hull_outer, max_connections=20)
add_edges_without_intersection(G, end, vertices_outer, hull_outer, max_connections=20)

# Define heuristic function for A* (Euclidean distance)
def heuristic(u, v):
    return np.linalg.norm(np.array(u) - np.array(v))

# Find the shortest path using A* algorithm
try:
    path = nx.astar_path(G, source, end, heuristic=heuristic, weight='weight')
    print("\nShortest path found by A* algorithm:")
    print(path)
except nx.NetworkXNoPath:
    print("\nNo path found from source to end.")

# Generate random rock-like points within the inner polytope
num_rock_points = 170
rock_points = []
while len(rock_points) < num_rock_points:
    point = np.random.normal(loc=np.mean(vertices_inner, axis=0), scale=scale_factor/3, size=3)
    if np.all(np.min(vertices_inner, axis=0) <= point) and np.all(point <= np.max(vertices_inner, axis=0)):
        rock_points.append(point)

rock_points = np.array(rock_points)

# Save rock points inside inner polytope to a file
# with open('rock.txt', 'w') as f:
#     for point in rock_points:
#         f.write(f"{point[0]}, {point[1]}, {point[2]}\n")

# Calculate points 0.1 units inside the inner polytope
distance_inside = 0.04
points_inside_inner_nodes = []
for vertex in vertices_inner:
    direction_to_center = center_outer - vertex
    direction_normalized = direction_to_center / np.linalg.norm(direction_to_center)
    new_point = vertex + distance_inside * direction_normalized
    points_inside_inner_nodes.append(new_point)

# Convert to a NumPy array for easier manipulation
points_inside_inner_nodes = np.array(points_inside_inner_nodes)

# Add these new points as nodes in the graph
for point in points_inside_inner_nodes:
    G.add_node(tuple(point))

# End timing
end_time = time.time()

# Calculate total runtime
total_time = end_time - start_time

# Display the total runtime
print(f"\nTotal runtime (excluding plotting): {total_time:.4f} seconds")




Shortest path found by A* algorithm:
[(-0.8, -0.8, -0.8), (-0.6892192113800709, -0.71, 0.14448833400878117), (-0.7090144449898003, -0.55, 0.4413598495511404), (-0.5673624854052105, -0.3900000000000001, 0.7252584436977084), (-0.3003060579947334, -0.22999999999999998, 0.9256977214683332), (0.03635382764311213, -0.07000000000000006, 0.9968843459577921), (0.37551252700982696, 0.08999999999999997, 0.9224371751283086), (0.8, 0.8, 0.8)]

Total runtime (excluding plotting): 0.0486 seconds
